# Buffer Tutorial

This tutorial explains how to use the `Buffer` class from microSCHC for bit-level binary data manipulation.

## Why Buffer?

The `Buffer` class is designed to handle binary data at the bit level, which is crucial for header compression. It addresses several challenges:

1. **Bit-level Precision**: The same byte sequence can represent different values depending on their bit length
2. **Padding Control**: Fields can be left-padded or right-padded
3. **Bit-level Operations**: Support for shifting, masking, and other binary operations

## Basic Usage

Let's start with some basic examples of creating buffers:

In [1]:
from microschc.binary.buffer import Buffer, Padding

# Create a buffer with explicit bit length
ipv6_address = Buffer(
    content=b'\x20\x01\x0d\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01',
    length=128,  # IPv6 address is 128 bits
    padding=Padding.LEFT  # Default padding
)

# Create a buffer for a 4-bit field
small_field = Buffer(
    content=b'\x06',  # Binary: 0110
    length=4,         # Only use 4 bits
    padding=Padding.LEFT
)

# Create a buffer for a 12-bit field
larger_field = Buffer(
    content=b'\x06',  # Binary: 0000 0000 0110
    length=12,        # Use 12 bits
    padding=Padding.LEFT
)

## Bit-level Operations

The `Buffer` class supports various bit-level operations:

In [2]:
# Bitwise operations
buffer1 = Buffer(content=b'\x0F', length=8)  # 0000 1111
buffer2 = Buffer(content=b'\xF0', length=8)  # 1111 0000

# AND operation
result_and = buffer1 & buffer2  # 0000 0000

# OR operation
result_or = buffer1 | buffer2   # 1111 1111

# XOR operation
result_xor = buffer1 ^ buffer2  # 1111 1111

# NOT operation
result_not = ~buffer1           # 1111 0000

## Shifting and Padding

Buffers can be shifted and padded in different ways:

In [3]:
# Create a buffer
buffer = Buffer(content=b'\x06', length=4)  # 0110

# Left shift by 2 bits
shifted_left = buffer.shift(-2)  # 1000

# Right shift by 1 bit
shifted_right = buffer.shift(1)  # 0011

# Change padding
right_padded = buffer.pad(Padding.RIGHT)  # 0110 with right padding

## Slicing and Indexing

You can access individual bits or ranges of bits:

In [4]:
# Create a buffer
buffer = Buffer(content=b'\x0F', length=8)  # 0000 1111

# Get a single bit
bit = buffer[3]  # Get the 4th bit (0-based indexing)

# Get a range of bits
bits = buffer[4:8]  # Get the last 4 bits

# Set bits
buffer[0:4] = Buffer(content=b'\x0F', length=4)  # Set first 4 bits

## Concatenation

Buffers can be concatenated using the `+` operator:

In [5]:
# Create two buffers
buffer1 = Buffer(content=b'\x0F', length=8)  # 0000 1111
buffer2 = Buffer(content=b'\xF0', length=8)  # 1111 0000

# Concatenate them
combined = buffer1 + buffer2  # 0000 1111 1111 0000

## Advanced Features

### Value Conversion


The `Buffer` class provides several ways to convert between different representations of binary data:

- **Integer Conversion**: Convert binary data to and from integers using the `value()` method with `type='unsigned int'`
- **Bytes Access**: Access the raw bytes content using the `content` property
- **Hex String**: Convert to hexadecimal string representation using Python's built-in `hex()` method
- **Binary String**: Convert to binary string representation by iterating over bits
- **From Integer**: Create a buffer from an integer by converting it to bytes
- **From Hex**: Create a buffer from a hexadecimal string using `bytes.fromhex()`

These conversions are essential when working with different data formats or when interfacing with other systems that use different binary representations.




In [11]:
buffer = Buffer(content=b'\x0F', length=8)  # 0000 1111

# Buffer as an integer
as_int = buffer.value(type='unsigned int')  # 15
print(f"value(type='unsigned int'): {as_int}")

# Buffer as bytes
as_bytes = buffer.content  # b'\x0F'
print(f"content as bytes: {as_bytes}")

# Convert to hex string
as_hex = buffer.content.hex()  # '0F'
print(f"content.hex(): {as_hex}")

# Convert to binary string
as_bin = ''.join(str(bit) for bit in buffer)  # '00001111'
print(f"binary string: {as_bin}")

# Create from integer
from_int = Buffer(content=(15).to_bytes(1, 'big'), length=8)  # 0000 1111
print(f"from integer: {''.join(str(bit) for bit in from_int)}")

# Create from hex string
from_hex = Buffer(content=bytes.fromhex('0F'), length=8)  # 0000 1111
print(f"from hex: {''.join(str(bit) for bit in from_hex)}")



value(type='unsigned int'): 15
content as bytes: b'\x0f'
content as hex: 0f

6-bit chunks:
Chunk 0: [--000000 ](6)
Chunk 1: [--010010 ](6)
Chunk 2: [--001101 ](6)
Chunk 3: [--000101 ](6)
Chunk 4: [--011001 ](6)
Chunk 5: [------11 ](2)


## Iterating, Chunking

The `chunks()` method allows you to split a buffer into smaller pieces of a specified size:

This is particularly useful when:
- Processing data in fixed-size blocks
- Implementing protocols that work with specific chunk sizes
- Analyzing binary data at different granularities

In [13]:

buffer = Buffer(content=b'\x01\x23\x45\x67', length=32)  # 00000001 00100011 01000101 01100111
sixbits_chunks = buffer.chunks(6)
fourbits_chunks = buffer.chunks(4)

print("6-bit chunks:")

for i, chunk in enumerate(sixbits_chunks):
    print(f"Chunk {i}: {chunk}")

print("4-bit chunks:")
for i, chunk in enumerate(fourbits_chunks):
    print(f"Chunk {i}: {chunk}")



6-bit chunks:
Chunk 0: [--000000 ](6)
Chunk 1: [--010010 ](6)
Chunk 2: [--001101 ](6)
Chunk 3: [--000101 ](6)
Chunk 4: [--011001 ](6)
Chunk 5: [------11 ](2)
4-bit chunks:
Chunk 0: [----0000 ](4)
Chunk 1: [----0001 ](4)
Chunk 2: [----0010 ](4)
Chunk 3: [----0011 ](4)
Chunk 4: [----0100 ](4)
Chunk 5: [----0101 ](4)
Chunk 6: [----0110 ](4)
Chunk 7: [----0111 ](4)



The `Buffer` class implements __get_item__ and __set_item__ methods which allows:

 - Access individual bits using indexing
 - Extract sub-buffers using slicing
 - Use negative indices to count from the end

These iteration methods provide flexible ways to process binary data, whether you need to examine individual bits or work with larger units.

In [18]:
buffer = Buffer(content=b'\x01\x23\x45\x67', length=32)

# slicing
buffer_4to12thbits = buffer[4:12]
print(buffer_4to12thbits)

# negative indexing
buffer_last_4bits = buffer[-4:]
print(buffer_last_4bits)




[00010010](8)
[----0111 ](4)
